In [2]:
import cv2
import numpy as np
from joblib import dump, load

In [3]:
# Importation de model
model = load('SVMmodel.joblib')

In [16]:
cap = cv2.VideoCapture(0)
rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50, 50))
#kernel = np.ones((5,5),np.uint8)
#rect = np.zeros((28,28), np.uint8)

#rectKernel = cv2.getStructuringElement(cv2.MORPH_RECT, (30, 30)) # a augmenter si "tremblement"
kernel_1 = cv2.getStructuringElement(cv2.MORPH_RECT, (10, 10))
dilate_k = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))


#Contrast parameter:
beta = -50
alpha = 1.7

while True:
    success, img = cap.read()
    imgContour = img.copy()
    resize = np.zeros((28,28), np.uint8)

    
    #frame modifier:
    imgGray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)
    contrast = 255-cv2.adaptiveThreshold(imgGray,500,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY,11,20)
    imgBlur = cv2.GaussianBlur(contrast,(3,3),0)
    morph_close = cv2.morphologyEx(imgBlur, cv2.MORPH_CLOSE, kernel_1)
    dilation = cv2.dilate(morph_close,dilate_k, iterations = 3)
    dilation_pred = cv2.dilate(morph_close,dilate_k, iterations = 1)
    blackhat = cv2.morphologyEx(dilation, cv2.MORPH_TOPHAT, rectKernel)
    blackhat_pred = cv2.morphologyEx(dilation_pred, cv2.MORPH_TOPHAT, rectKernel)
    imgCanny = cv2.Canny(blackhat, 500, 500)
    
    
    
    #Get contours:
    contours,hierarchy = cv2.findContours(imgCanny,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_NONE)
    
    
    c=0
    for cnt in contours:
        area = cv2.contourArea(cnt)
        
        #Draw Contours:
        if area > 1500 and c < 1:
            cv2.drawContours(imgContour, cnt, -1, (255,0,0),3)
            peri = cv2.arcLength(cnt,True)
            approx = cv2.approxPolyDP(cnt,0.02*peri,True)
            x, y, w, h = cv2.boundingRect(approx)
            x=x-10
          
            #Draw Rectangle:
            max_size = max(h, w)
            margin = round(max_size / 6)
            cv2.rectangle(imgContour, (x - margin, y - margin),
            (x + max_size + margin, y + max_size + margin), (0, 255, 0), 2)
            rect = blackhat_pred[y - margin:y + max_size + margin, x - margin:x + max_size + margin]
            
                
            #Get Predict
            if rect.shape[0] != 0 and rect.shape[1] != 0:
                
                resize = cv2.resize(rect, (28,28))
                X = resize.reshape(1,-1)
                y_pred = model.predict(X)
                c+=1
                #Print Predict:
                cv2.putText(imgContour,str(y_pred[0]),(x-30,y-30),cv2.FONT_HERSHEY_COMPLEX,1,(0,0,0),2)
    
    
    #Resize predict:
    predictImg = cv2.resize(resize,(224,224), interpolation=cv2.INTER_AREA)
    
    
    # Windows:
    cv2.imshow("Result",imgContour)
    cv2.imshow("Result2",predictImg)
    
    #Quit:
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
        
        
cap.release()
cv2.destroyAllWindows()